In [1]:
# import librarie
from util.generate_moa_stream import run_generate_grad_stream_moa, plot_anomaly, plot_stream_drift
from util.combine_stream import get_arff_data_labels, find_anomaly_intervals
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys

In [2]:
# declare constant values
MOA_FILEPATH = '/Users/tammyz/Desktop/3H03/moa-release-2023.04.0/lib' # path to MOA lib directory
SOURCE_DIR = "data/benchmark/ECG" # directory for dataset of interest
OUTPUT_DIR = "data/synthetic" # directory for storing generated ARFF files

In [3]:
# list files and select 6 for combining
files = os.listdir(SOURCE_DIR)
files.sort()
selected_streams = files[2:8]

# find and store the anomaly intervals for all files
anom_ints = []
for s in selected_streams:
    _, y = get_arff_data_labels(os.path.join(SOURCE_DIR,s))
    anom_ints.append(find_anomaly_intervals(y))

# print details of selected streams
df = pd.read_csv(f'{SOURCE_DIR}/description.csv')
df.loc[df.filename.apply(lambda x: x in selected_streams)]

,filename,len,num_anomalies,total_anom_cover,avg_anomaly_len,percent_anomalies
2,MBA_ECG14046_data_10.arff,229900,107,8130,75.981308,0.035363
3,MBA_ECG14046_data_11.arff,229900,165,12746,77.248485,0.055441
4,MBA_ECG14046_data_12.arff,229900,169,14318,84.721893,0.062279
5,MBA_ECG14046_data_13.arff,229900,105,8265,78.714286,0.035950
6,MBA_ECG14046_data_14.arff,229900,128,10159,79.367188,0.044189
7,MBA_ECG14046_data_15.arff,229900,170,13066,76.858824,0.056833


In [10]:
# define parameters to generate drift
length = 229900
n_drift = 33
p_drift = 0.40
p_drift_after = 0.5
max_stream = len(selected_streams)-1
anom_ints = anom_ints
trial_name = 'p_drift'

In [11]:
output_path, drift_label, streams, positions, seq_drift_after = run_generate_grad_stream_moa(
    selected_streams,
    p_drift, 
    n_drift, 
    length, 
    p_drift_after, 
    max_stream, 
    anom_ints,
    trial_name
)

Generating splits...
Getting partitions...
Getting order of drifts coming after anomaly...
Getting drift center positions...
Getting stream file cuts...
Creating intermediate files...
Recursively generating MOA command...
Drift filename:  ECG_grad_p25_n22_a54
Streams: [0, 1, 2, 5, 2, 1, 5, 1, 3, 0, 5, 1, 2, 4, 2, 4, 2, 0, 5, 2, 5, 0, 3]
Positions: [24855, 38097, 46615, 55440, 63468, 77781, 84927, 90910, 102510, 113770, 118175, 126424, 136280, 138518, 150268, 161921, 172258, 176108, 189719, 198067, 215005, 225254]
w_drift: [3775, 2629, 4289, 1330, 466, 966, 4125, 664, 2649, 1974, 1401, 4579, 1896, 2803, 4146, 4577, 2769, 105, 2701, 783, 4215, 4687]
Running terminal command...



{M}assive {O}nline {A}nalysis
Version:  23.04 April 2023
Copyright: (C) 2007-2023 University of Waikato, Hamilton, New Zealand
Web: http://moa.cms.waikato.ac.nz/

                                                                               
Task completed in 0.97s (CPU time)



Stream written to ARFF file /Users/tammyz/Desktop/3H03/AnomalyDriftDetection/data/synthetic/p_drift/ECG_grad_p25_n22_a54.arff
Generating drift labels...
value:  [-111] | index:  12
w_drift: [4579, 1896, 2803]
positions: [126424, 136280, 138518]
